In [ ]:
import numpy as np
import math
from scipy.optimize import linear_sum_assignment

def generate_curved_scenario(n_frames=100, noise_params=None):
    """
    Создание Ground Truth (GT) и зашумленных детекций.
    
    noise_params: dict {'sigma_pos': float, 'sigma_yaw': float, 'sigma_dim': float}
    
    Returns:
        gt_data: list of lists. Каждый элемент списка кадр.
                 Внутри кадра: [{'id': int, 'box': np.array([x, y, z, yaw, w, h])}, ...]
        detections: list of lists. Внутри кадра: [np.array([x, y, z, yaw, w, h]), ...]
    """
    if noise_params is None:
        noise_params = {'sigma_pos': 0.15, 'sigma_yaw': 0.05, 'sigma_dim': 0.1}
    
    gt_data = []
    detections = []
    
    # Инициализация параметров объектов
    for t in range(n_frames):
        frame_gt = []
        frame_dets = []
        
        time = t * 0.1  # dt = 0.1s
        
        # Sine Wave 
        x0 = 0 + 1.5 * time
        y0 = 5 + 2.0 * np.sin(0.5 * time)
        z0 = 0
        yaw0 = np.arctan2(2.0 * 0.5 * np.cos(0.5 * time), 1.5) 
        w0 = 2.0 + 0.2 * np.sin(time)
        h0 = 4.0
        
        box0 = np.array([x0, y0, z0, yaw0, w0, h0])
        frame_gt.append({'id': 0, 'box': box0})
        
        # Circle 
        radius = 8.0
        omega = 0.2
        x1 = 10 + radius * np.cos(omega * time)
        y1 = 10 + radius * np.sin(omega * time)
        z1 = 0
        yaw1 = omega * time + np.pi/2
        w1 = 3.0
        h1 = 1.5 + 0.5 * np.cos(time)
        
        box1 = np.array([x1, y1, z1, yaw1, w1, h1])
        frame_gt.append({'id': 1, 'box': box1})
        
        # Diagonal 
        x2 = -5 + 1.0 * time
        y2 = -5 + 1.0 * time
        z2 = 0
        yaw2 = np.pi/4
        w2 = 2.0
        h2 = 2.0
        
        box2 = np.array([x2, y2, z2, yaw2, w2, h2])
        frame_gt.append({'id': 2, 'box': box2})
        
        # Создание детекций (GT + Noise)
        for obj in frame_gt:
            noise_pos = np.random.normal(0, noise_params['sigma_pos'], 3)
            noise_pos[2] = noise_pos[2] * 0.2 
            noise_yaw = np.random.normal(0, noise_params['sigma_yaw'])
            noise_dim = np.random.normal(0, noise_params['sigma_dim'], 2)
            
            det_box = obj['box'].copy()
            det_box[0:3] += noise_pos
            det_box[3] += noise_yaw
            det_box[4:6] += noise_dim
            
            frame_dets.append(det_box)
        
        # Перемешивание
        np.random.shuffle(frame_dets)
        
        gt_data.append(frame_gt)
        detections.append(frame_dets)
        
    return gt_data, detections

In [ ]:
def compute_metrics(gt_data, tracker_history, dist_threshold=2.0):

    # Метрики на основе истории трекера и GT
    
    total_frames = len(gt_data)
    total_gt_objects = 0
    total_matches = 0
    position_errors = []
    
    for t in range(total_frames):
        gts = gt_data[t] 
        trks = tracker_history[t] 
        
        if len(gts) == 0:
            continue
            
        total_gt_objects += len(gts)
        
        if len(trks) == 0:
            continue
            
        # Матрица расстояний 
        cost_matrix = np.zeros((len(gts), len(trks)))
        for i, gt in enumerate(gts):
            for j, trk in enumerate(trks):
                dist = np.linalg.norm(gt['box'][0:2] - trk['box'][0:2])
                cost_matrix[i, j] = dist
                
        # Сопоставление (Венгерский) для оценки
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        
        for r, c in zip(row_ind, col_ind):
            dist = cost_matrix[r, c]
            if dist < dist_threshold:
                total_matches += 1
                position_errors.append(dist)
                
    mae = np.mean(position_errors) if len(position_errors) > 0 else 0.0
    matching_accuracy = total_matches / total_gt_objects if total_gt_objects > 0 else 0.0
    
    return {
        'MAE_Pos': mae,
        'Matching_Accuracy': matching_accuracy,
        'Total_Frames': total_frames
    }

# Проверка размерностей
gt_test, det_test = generate_curved_scenario(n_frames=5)
print(f"Frames generated: {len(gt_test)}")
print(f"Objects in frame 0: {len(gt_test[0])}")
print(f"Detection format sample: {det_test[0][0]}") 